## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-09-05-41-52 +0000,nypost,"Ex-Marlins draft pick sets flags on fire, tips...",https://nypost.com/2025/10/09/us-news/washingt...
1,2025-10-09-05-33-52 +0000,nyt,"With Mideast Deal, Trump Is on the Brink of a ...",https://www.nytimes.com/2025/10/08/us/politics...
2,2025-10-09-05-22-34 +0000,bbc,China tightens export rules for crucial rare e...,https://www.bbc.com/news/articles/ckgzl0nwvd7o...
3,2025-10-09-05-20-55 +0000,nypost,California mayor pens bizarre letter to local ...,https://nypost.com/2025/10/09/us-news/south-la...
4,2025-10-09-05-07-13 +0000,nypost,US diplomat fired over relationship with woman...,https://nypost.com/2025/10/09/us-news/us-state...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,54
49,gaza,22
132,new,21
6,fire,17
472,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
77,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...,167
30,2025-10-09-02-51-03 +0000,startribune,Israel and Hamas agree to part of Trump’s Gaza...,https://www.startribune.com/hamas-and-israel-e...,146
34,2025-10-09-02-27-31 +0000,bbc,Trump says Israel and Hamas 'sign off' on firs...,https://www.bbc.com/news/articles/ce80rmq3g5qo...,141
14,2025-10-09-04-23-07 +0000,nyt,President Trump Says Israel and Hamas Agree to...,https://www.nytimes.com/2025/10/08/world/middl...,120
192,2025-10-08-18-05-37 +0000,nypost,"Trump mulls weekend trip to Egypt, Gaza to sea...",https://nypost.com/2025/10/08/world-news/israe...,117


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
77,167,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...
267,79,2025-10-08-13-33-06 +0000,nypost,Ex-FBI Director James Comey insists he’s innoc...,https://nypost.com/2025/10/08/us-news/ex-fbi-d...
161,70,2025-10-08-20-00-18 +0000,latimes,'Why wasn't the fire put out on the 1st?' Caus...,https://www.latimes.com/california/story/2025-...
121,65,2025-10-08-21-33-46 +0000,nypost,Andrew Cuomo would scrap plan to close Rikers ...,https://nypost.com/2025/10/08/us-news/andrew-c...
92,60,2025-10-08-22-51-00 +0000,wsj,President Trump’s declaration that Illinois Go...,https://www.wsj.com/politics/policy/trump-prit...
78,53,2025-10-08-23-16-00 +0000,wsj,Senior GOP lawmakers have quietly advised the ...,https://www.wsj.com/politics/policy/republican...
165,41,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Chica...,https://www.latimes.com/politics/story/2025-10...
36,39,2025-10-09-02-06-44 +0000,nypost,Transportation Secretary Sean Duffy urges air ...,https://nypost.com/2025/10/08/us-news/transpor...
137,36,2025-10-08-20-56-43 +0000,nypost,Supreme Court likely to favor GOP challenge to...,https://nypost.com/2025/10/08/us-news/supreme-...
334,36,2025-10-08-09-00-00 +0000,wsj,The nation’s top four wind power states all le...,https://www.wsj.com/politics/policy/trumps-win...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
